In [1]:
# Importations

import requests
import json
import numpy as np
import pandas as pd
from time import sleep
from urllib.parse import urlencode
from math import sin, cos, sqrt, atan2, radians

In [2]:
# API KEY from Google Maps API

API_KEY = ""


# Perimeter coordinates of the study area

LAT1 = 40.448299
LAT2 = 40.431397
LONG1 = -3.700018
LONG2 = -3.722460


# Search categories list

CATEGORIES = ["airport", "bar", "bus_station", "cafe", "church", "city_hall",
              "convenience_store", "department_store", "hospital", 
              "light_rail_station", "local_government_office", "night_club",
              "park", "parking", "pharmacy", "post_office", "primary_school", 
              "restaurant", "school", "secondary_school", "shopping_mall", 
              "stadium", "subway_station", "synagogue", "taxi_stand", 
              "tourist_attraction", "train_station", "transit_station", "university"]

In [3]:
# Big categories (2x2 searches)

CATEGORIES_BIG = ["airport", "city_hall", "light_rail_station", "post_office", 
                  "shopping_mall", "stadium", "train_station", "university"]


# Medium categories (4x4 searches)

CATEGORIES_MEDIUM = ["bus_station", "church", "department_store", "hospital", 
                     "local_government_office", "night_club", "park", "parking", 
                     "primary_school", "school", "secondary_school", 
                     "subway_station", "synagogue", "taxi_stand", 
                     "tourist_attraction", "train_station", "transit_station"]


# Small categories (8x8 searches)

CATEGORIES_SMALL = ["bar", "cafe", "convenience_store", "pharmacy", "restaurant"]

In [4]:
def calculate_distance(point1, point2):
    
    """
    Calculate the distance (in kilometers) between two points given
    by their latitude and longitude coordinates.
    """
    
    # Earth's aproximated radio in kilometers
    R = 6373.0
    
    # Coordinates need to be in radians
    lat1, lng1 = radians(float(point1[0])), radians(float(point1[1]))
    lat2, lng2 = radians(float(point2[0])), radians(float(point2[1]))
    
    # Difference between latitudes and longitudes
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    
    # Distance estimation
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [5]:
def define_search_geometry(lats, lngs, n_lat=8, n_lng=8):
    
    """
    Determine the coordinates and radius needed to get a search in the
    study area (delimited by its latitudes and longitudes). The number
    of searches are given by the n_lat and n_lng parameters.
    """
    
    # Latitudes' geometry
    lat1, lat2 = lats[0], lats[1]
    lat_separation = (lat2 - lat1)/n_lat
    lat_geometry = np.array(lat1) + (np.arange(n_lat) + 0.5) * lat_separation
    
    # Longitudes' geometry
    lng1, lng2 = lngs[0], lngs[1]
    lng_separation = (lng2 - lng1)/n_lng
    lng_geometry = np.array(lng1) + (np.arange(n_lng) + 0.5) * lng_separation
    
    # Resultant geometry
    coordinates = [(lat, lng) for lat in lat_geometry for lng in lng_geometry]
    
    lat_sep_distance = calculate_distance([lat1, lng1], [lat1 + lat_separation, lng1])
    lng_sep_distance = calculate_distance([lat1, lng1], [lat1, lng1 + lng_separation])
    radius = sqrt(lat_sep_distance**2 + lng_sep_distance**2)/2 * 1000
    
    geometry = {"coordinates": coordinates, "radius": radius}
    
    return geometry

In [6]:
def generate_url(api_key, coordinates, radius, keyword=None, placetype=None, output="json"):
    
    """
    Generate an URL with the Nearby Search tool from Google Maps Places API.
    """
    
    # Endpoint generation
    endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{output}"
    
    # Parameters generation and encoding
    parameters = {"key": api_key, 
                  "location": f"{coordinates[0]},{coordinates[1]}", 
                  "radius": radius,}
    
    if keyword:
        parameters["keyword"] = keyword
    if placetype:
        parameters["type"] = placetype
    
    url_parameters = urlencode(parameters)
    
    # URL generation
    url = f"{endpoint}?{url_parameters}"
    
    return url

In [7]:
def get_url(url):
    
    """
    Get a given URL in json format.
    """
    
    return requests.get(url).json()

In [8]:
def write_json(filename, json_data):
    
    """
    Write data in a file in json format.
    """
    
    with open(filename, "a") as file:
        file.write(json.dumps(json_data))
        file.write("\n\n")

In [9]:
def read_json(filename):
    
    """
    Read data from a file in json format.
    """
    
    with open(filename, "r") as file:
        json_data = file.read()
        
    return json.loads(json_data)

In [10]:
def search_nearby_places(api_key, filename, coordinates, radius, 
                         keyword=None, placetype=None, output="json"):
    
    """
    Get a search with the Nearby Search tool by a keyword or a placetype.
    """
    
    # Generate the URL direction
    url = generate_url(api_key, coordinates, radius, keyword=keyword, 
                       placetype=placetype, output=output)
    
    # Access to the URL
    rjson = get_url(url)
    
    # Write the results of the extraction in a file
    write_json(filename, rjson)
    
    # Check if there exists another page
    while "next_page_token" in rjson.keys():
        sleep(2)             # A needed wait the token to work properly
        rjson = get_url(url + f"&pagetoken={rjson['next_page_token']}")
        write_json(filename, rjson)

In [11]:
def extract_nearby_places(geometry_params, search_params, filename):
    
    """
    Generalize the data extraction including search geometry. 
    """
    
    # Define the search geometry
    geometry = define_search_geometry(lats = geometry_params["lats"], 
                                      lngs = geometry_params["lngs"], 
                                      n_lat = geometry_params["n_lat"], 
                                      n_lng = geometry_params["n_lng"])
    
    # Search nearby places with the desired parameters
    for i in range(len(geometry["coordinates"])):
        search_nearby_places(api_key = search_params["api_key"],
                             filename = filename,
                             coordinates = geometry["coordinates"][i], 
                             radius = geometry["radius"], 
                             keyword = search_params["keyword"],
                             placetype = search_params["placetype"], 
                             output="json")
        
        print("Done.")

In [ ]:
### EXAMPLE ###

In [12]:
# Geometric and search parameters

geometry_params = {"lats": [LAT1, LAT2], "lngs": [LONG1, LONG2], "n_lat": 2, "n_lng": 2}
search_params = {"api_key": API_KEY, "keyword": "geriatria", "placetype": None}


# Output file name

filename = "example1"

In [13]:
# The search

extract_nearby_places(geometry_params, search_params, filename)

Done.
Done.
Done.
Done.


In [7]:
def generate_url_2(api_key, text, coordinates, radius, inputtype = "textquery", output="json"):
    
    """
    Generate an URL with the Search Place tool from Google Maps Places API.
    """
    
    # Endpoint generation
    endpoint = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/{output}"
    
    # Parameters generation and encoding
    lat, lng = coordinates[0], coordinates[1]
    parameters = {"key": api_key, 
                  "input": text,
                  "inputtype": inputtype,
                  "locationbias": f"circle:{radius}@{lat},{lng}", 
                  "radius": radius,
                 }
    url_parameters = urlencode(parameters)
    
    # URL generation
    url = f"{endpoint}?{url_parameters}"
    
    return url

In [15]:
def search_places_by_text(api_key, text, coordinates, radius, filename, output="json"):
    
    """
    Get a search with the Search Place tool by a text.
    """
    
    # Generate the URL direction
    url = generate_url_2(api_key, text, coordinates, radius)
    
    # Access to the URL
    rjson = get_url(url)
    
    # Write the results of the extraction in a file
    write_json(filename, rjson)
    
    # Check if there exists another page
    while "next_page_token" in rjson.keys():
        sleep(2)
        rjson = get_url(url + f"&pagetoken={rjson['next_page_token']}")
        write_json(filename, rjson)

In [17]:
def extract_places_by_text(geometry_params, search_params, filename):
    
    """
    Generalize the data extraction including search geometry. 
    """
    
    # Define the search geometry
    geometry = define_search_geometry(lats = geometry_params["lats"], 
                                      lngs = geometry_params["lngs"], 
                                      n_lat = geometry_params["n_lat"], 
                                      n_lng = geometry_params["n_lng"])
    
    # Search a place with the desired parameters
    for i in range(len(geometry["coordinates"])):
        search_places_by_text(api_key = search_params["api_key"], 
                              coordinates = geometry["coordinates"][i], 
                              radius = geometry["radius"], 
                              text = search_params["text"], 
                              filename = filename, 
                              output="json")
        
        print("Done.")

In [ ]:
### EXAMPLE ###

In [ ]:
# Geometric and search parameters

geometry_params = {"lats": [LAT1, LAT2], "lngs": [LONG1, LONG2], "n_lat": 2, "n_lng": 2}
search_params = {"api_key": API_KEY, "text": "dia"}


# Output file name

filename = "example2"

In [19]:
# The search

extract_places_by_text(geometry_params, search_params_2, filename)

Done.
Done.
Done.
Done.
